In [5]:
import subprocess
from pydub import AudioSegment
import math

def extract_audio_from_video(vidoe_path, audio_path):
  command = ["ffmpeg", "-i", vidoe_path, "-vn", audio_path]
  subprocess.run(command)

# extract_audio_from_video("files/video.mp4", "files/audio.mp3")

def cut_audio_in_chunks(audio_path, chunk_size_in_minute, chunks_path, offset_in_sec):
  track = AudioSegment.from_mp3(audio_path)
  chunk_len = chunk_size_in_minute * 60 * 1000
  chunk= math.ceil(len(track) / chunk_len)
  for i in range(chunk):
    start_time = (offset_in_sec*1000) + (i * chunk_len)
    end_time = (offset_in_sec*1000) + ((i+1) * chunk_len)
    chunk = track[start_time:end_time]
    chunk.export(f"./{chunks_path}/chunk_{i}.mp3", format="mp3")

cut_audio_in_chunks("./files/audio.mp3", 10, "files/chunks", 88)


In [7]:
import openai
import glob

def transcript_chunks(chunk_path, dest):
  files = glob.glob(f"{chunk_path}/*.mp3")
  final_transcript = ""
  for file in files:
    with open(file, "rb") as audio_file:
      transcript = openai.Audio.transcribe("whisper-1", audio_file)
      final_transcript += transcript["text"]
  with open(dest, "w") as file:
    file.write(final_transcript)

transcript_chunks("./files/chunks", "./files/transcript.txt")